# CS440 Assignment 1 - Search

## Part 1.1

## State Representation

In [45]:
class Node:
    def __init__(self, rowPos, colPos, value):
        self.rowPos = rowPos
        self.colPos = colPos
        self.value = value
        self.isWall = False  
        self.isGoal = False
        self.distance = 0

In [270]:
from collections import deque
from IPython.display import clear_output
import sys
import time

goal_dict = {}

class Maze:
    def __init__(self, filename):     
        self.goals = []
        self.boxes = []
        self.goalCounter = '0'
        self.start = None
        self.rowsInMaze = 0
        self.colsInMaze = 0
        self.colsList = []
        self.path = []
        self.explored = {}
        self.nodesExpanded = 0
        
        # Read maze from file
        mazeObject = open(filename, 'r')
        rows = 0
        
        # Loop through rows in Maze
        for line in mazeObject:
            rowList = []
            cols = 0
            
            # Loop through columns in Maze
            for char in line[:-1]:
                node = Node(rows, cols, char)
                
                rowList.append(node)
                
                # Look for starting point
                if char == 'P':
                    self.start = node
                    
                # Look for goals
                if char == '.':
                    node.isGoal = True
                    self.goals.append(node)
                    
                # Look for walls
                if char == '%':
                    node.isWall = True
                    
                # Look for boxes
                if char == 'b':
                    self.boxes.append(node)
                
                # Look for boxes
                if char == 'B':
                    self.boxes.append(node)
                    self.goals.append(node)
                    node.isGoal = True
                
                cols = cols + 1
            rows = rows + 1
        
            self.colsList.append(rowList)
            self.colsInMaze = len(rowList)
        
        self.rowsInMaze = rows
    
    def runSokobanSearch(self, searchType):
        startTime = time.time()
        state = State(self.start, self.goals[:], self.boxes[:])
        self.explored[hash(state)] = state
        
        if searchType == 'bfs':
            self.frontier = deque([])
        else:
            self.frontier = []
        
        # Initialize frontier based on starting node
        addToFrontierSokoban(state, self)
        
        while self.frontier != []:
            if searchType == 'bfs':
                nextState = self.frontier.popleft()
                state = sokobanMove(nextState, self)
        
            if searchType == 'astarv2': 
                index = 0
                total_distance = sys.maxsize
                for i in range(0, len(self.frontier)):
                    if self.frontier[i].distance <= total_distance:
                        total_distance = self.frontier[i].distance
                        index = i
                nextState = self.frontier.pop(index)
                state = sokobanMove(nextState, self)
            
            # Check if no goals exist
            if (sokobanGoalTest(state.goals)):
                endTime = time.time()
                print(endTime - startTime)
                path = []
                while state.previous != None:
                    path.append(state)
                    state = state.previous
                print("Nodes Expanded: " + str(self.nodesExpanded))
                print("Nodes in Path: " + str(len(path)))
                self.printMaze()
                return
    
    def runSearch(self, searchType):   
        startTime = time.time()
        state = State(self.start, self.goals[:], self.boxes[:])
        self.explored[hash(state)] = state
        
        if searchType == 'bfs':
            self.frontier = deque([])
        else:
            self.frontier = []

        # Initialize frontier based on starting node
        addToFrontier(state, self)        

        while self.frontier != []:
            # Search Algorithms
            if searchType == 'dfs':
                nextState = self.frontier.pop()
                state = move(nextState, self)
                
            if searchType == 'bfs':
                nextState = self.frontier.popleft()
                state = move(nextState, self)

            if searchType == 'gbfs':
                distance = sys.maxsize     
                index = 0
                for i in range(0, len(self.frontier)):
                    for goal in state.goals:
                        if abs(self.frontier[i].currentRowPos - goal.rowPos) + abs(self.frontier[i].currentColPos - goal.colPos) <= distance:
                            distance = abs(self.frontier[i].currentRowPos - goal.rowPos) + abs(self.frontier[i].currentColPos - goal.colPos)
                            index = i
                nextState = self.frontier.pop(index)
                state = move(nextState, self)

            if searchType == 'astar':
                distance = sys.maxsize     
                index = 0
                for i in range(0, len(self.frontier)):
                    for goal in state.goals:
                        distance_to_goal = abs(self.frontier[i].currentRowPos - goal.rowPos) + abs(self.frontier[i].currentColPos - goal.colPos)
                        distance_from_start = self.frontier[i].cost

                        if distance_to_goal + distance_from_start <= distance:
                            distance = distance_to_goal + distance_from_start
                            index = i

                nextState = self.frontier.pop(index)
                state = move(nextState, self)
                
            if searchType == 'astarv2': 
                index = 0
                total_distance = sys.maxsize
                for i in range(0, len(self.frontier)):
                    if self.frontier[i].distance <= total_distance:
                        total_distance = self.frontier[i].distance
                        index = i
                #print("Distance: " + str(self.frontier[index].distance))
                nextState = self.frontier.pop(index)
                state = move(nextState, self)

            # Check if no goals exist
            if (goalTest(state.goals)):
                endTime = time.time()
                print(endTime - startTime)
                path = []
                if len(self.goals) > 10:
                    self.goalCounter = chr(ord('a') + len(self.goals) - 10)
                else:
                    self.goalCounter = chr(ord('0') + len(self.goals))
                while state.previous != None:
                    if state.getCurrentNode(self).isGoal and state.getCurrentNode(self).value == '.':
                        state.getCurrentNode(self).value = self.goalCounter
                        self.goalCounter = incrementGoalCounter(self.goalCounter)
                    if state.getCurrentNode(self).value == ' ':
                        state.getCurrentNode(self).value = '.'
                    path.append(state)
                    state = state.previous
                print("Nodes Expanded: " + str(self.nodesExpanded))
                print("Nodes in Path: " + str(len(path)))
                self.printMaze()
                return

    def printMaze(self):          
        for row in self.colsList:
            line = ""
            for node in row:
                line += node.value
            print(line)

In [180]:
import sys
def furthestDistance(state):
    verticalMax = 0
    verticalMin = sys.maxsize
    horizontalMax = 0
    horizontalMin = sys.maxsize
    shortestPath = sys.maxsize
    shortestGoalRow = state.currentRowPos
    shortestGoalCol = state.currentColPos
    
    for goal in state.goals:
        if goal.rowPos >= verticalMax:
            verticalMax = goal.rowPos
        if goal.colPos >= horizontalMax:
            horizontalMax = goal.colPos
        if goal.rowPos <= verticalMin:
            verticalMin = goal.rowPos
        if goal.colPos <= horizontalMin:
            horizontalMin = goal.colPos
        if abs(state.currentRowPos - goal.rowPos) + abs(state.currentColPos - goal.colPos) <= shortestPath:
            shortestPath = abs(state.currentRowPos - goal.rowPos) + abs(state.currentColPos - goal.colPos)
            shortestGoalRow = goal.rowPos
            shortestGoalCol = goal.colPos

    closestBorderVertical = abs(shortestGoalRow - verticalMax)
    if closestBorderVertical >= abs(shortestGoalRow - verticalMin):
        closestBorderVertical = abs(shortestGoalRow - verticalMin)
    closestBorderHorizontal = abs(shortestGoalCol - horizontalMax)
    if closestBorderHorizontal >= abs(shortestGoalCol - horizontalMin):
        closestBorderHorizontal = abs(shortestGoalCol - horizontalMin)
    return abs(verticalMax - verticalMin) + abs(horizontalMax - horizontalMin) + shortestPath + closestBorderVertical + closestBorderHorizontal

In [51]:
class State:
    def __init__(self, currentNode, goals, boxes):
        self.goals = goals
        self.boxes = boxes
        self.currentRowPos = currentNode.rowPos
        self.currentColPos = currentNode.colPos
        self.cost = 0
        self.previous = None
    
    def __hash__(self):
        goalString = ""
        boxString = ""
        for goal in self.goals:
            goalString += str(goal.rowPos) + str(goal.colPos)
        if self.boxes != None:
            for box in self.boxes:
                boxString += str(box.rowPos) + str(box.colPos)
        return hash((str(self.currentRowPos), str(self.currentColPos), goalString, boxString))
        
    def getCurrentNode(self, maze):
        return maze.colsList[self.currentRowPos][self.currentColPos]

## Transition Model

In [52]:
def hash_goals(goals):
    hash_string = ""
    for goal in goals:
        hash_string += str(goal.rowPos) + "," + str(goal.colPos)
    return hash_string

In [260]:
def move(state, maze):    
    # Check if state is goal
    if state.getCurrentNode(maze) in state.goals:
        tempGoals = state.goals[:]
        tempGoals.remove(state.getCurrentNode(maze))
        newState = State(state.getCurrentNode(maze), tempGoals[:], state.boxes[:])
        newState.previous = state
        newState.cost = state.cost
        distance = sys.maxsize
        distance_remaining = furthestDistance(newState)       
        newState.distance = distance_remaining + newState.cost
        maze.explored[hash(newState)] = newState   
        addToFrontier(newState, maze)
        return newState
    elif hash(state) in maze.explored:
        return state
    else:   
        addToFrontier(state, maze)
        maze.explored[hash(state)] = state   
        return state
    
def sokobanMove(state, maze):
    if hash(state) in maze.explored:
        return state
    else:
        addToFrontierSokoban(state, maze)
        maze.explored[hash(state)] = state
        return state

In [228]:
from IPython.display import clear_output
import sys
def addToFrontier(state, maze): 
    maze.nodesExpanded += 1
    #print(maze.nodesExpanded)           
    
    # Top Node
    if state.currentRowPos > 1 and not maze.colsList[state.currentRowPos - 1][state.currentColPos].isWall:  
        newState = State(maze.colsList[state.currentRowPos - 1][state.currentColPos], state.goals[:], state.boxes[:])
        newState.cost = state.cost + 1
        newState.previous = state
        distance_remaining = furthestDistance(newState)
                
        newState.distance = distance_remaining + newState.cost
        maze.frontier.append(newState)
        
    # Bottom Node
    if state.currentRowPos < maze.rowsInMaze - 1 and not maze.colsList[state.currentRowPos + 1][state.currentColPos].isWall:
        newState = State(maze.colsList[state.currentRowPos + 1][state.currentColPos], state.goals[:], state.boxes[:])
        newState.cost = state.cost + 1
        newState.previous = state
        distance_remaining = furthestDistance(newState)
                
        newState.distance = distance_remaining + newState.cost
        maze.frontier.append(newState)
        
        
    # Left Node
    if state.currentRowPos > 1 and not maze.colsList[state.currentRowPos][state.currentColPos - 1].isWall:
        newState = State(maze.colsList[state.currentRowPos][state.currentColPos - 1], state.goals[:], state.boxes[:])
        newState.cost = state.cost + 1
        newState.previous = state
        distance_remaining = furthestDistance(newState)
                
        newState.distance = distance_remaining + newState.cost
        maze.frontier.append(newState)
        
    # Right Node
    if state.currentColPos < maze.colsInMaze - 1 and not maze.colsList[state.currentRowPos][state.currentColPos + 1].isWall:
        newState = State(maze.colsList[state.currentRowPos][state.currentColPos + 1], state.goals[:], state.boxes[:])
        newState.cost = state.cost + 1
        newState.previous = state
        distance_remaining = furthestDistance(newState)
                
        newState.distance = distance_remaining + newState.cost
        maze.frontier.append(newState)
        
    #printedFrontier = []
    #for state in maze.frontier:
        #printedFrontier.append(str(len(state.goals)))
    #print(printedFrontier)
    #maze.printMaze()
    ##input("Press enter to continue")

In [281]:
from IPython.display import clear_output
import sys
def addToFrontierSokoban(state, maze): 
    maze.nodesExpanded += 1
    #print(maze.nodesExpanded)           
    
    # Top Node
    currentBox = None
    nextBox = None
    if not maze.colsList[state.currentRowPos - 1][state.currentColPos].isWall:  
        for box in state.boxes:
            if box.rowPos == state.currentRowPos - 1 and box.colPos == state.currentColPos:
                currentBox = box
        for box in state.boxes:
            if box.rowPos == state.currentRowPos - 2 and box.colPos == state.currentColPos:
                nextBox = box
        if currentBox != None:
            if nextBox == None:
                currentBox.rowPos -= 1
                newState = State(maze.colsList[state.currentRowPos - 1][state.currentColPos], state.goals[:], state.boxes[:])
                newState.cost = state.cost + 1
                newState.previous = state
                distance_remaining = furthestDistance(newState)
                
                newState.distance = distance_remaining + newState.cost
                #print(str(newState.currentRowPos) + "," + str(newState.currentColPos))
                maze.frontier.append(newState)
            else:
                pass
        else:  
            newState = State(maze.colsList[state.currentRowPos - 1][state.currentColPos], state.goals[:], state.boxes[:])
            newState.cost = state.cost + 1
            newState.previous = state
            distance_remaining = furthestDistance(newState)

            newState.distance = distance_remaining + newState.cost
            maze.frontier.append(newState)
     
    # Bottom Node
    currentBox = None
    nextBox = None
    if not maze.colsList[state.currentRowPos + 1][state.currentColPos].isWall:  
        for box in state.boxes:
            if box.rowPos == state.currentRowPos + 1 and box.colPos == state.currentColPos:
                currentBox = box
        for box in state.boxes:
            if box.rowPos == state.currentRowPos + 2 and box.colPos == state.currentColPos:
                nextBox = box
        if currentBox != None:
            if nextBox == None:
                currentBox.rowPos += 1
                newState = State(maze.colsList[state.currentRowPos + 1][state.currentColPos], state.goals[:], state.boxes[:])
                newState.cost = state.cost + 1
                newState.previous = state
                distance_remaining = furthestDistance(newState)
                
                newState.distance = distance_remaining + newState.cost
                #print(str(newState.currentRowPos) + "," + str(newState.currentColPos))
                maze.frontier.append(newState)
            else:
                pass
        else:  
            newState = State(maze.colsList[state.currentRowPos + 1][state.currentColPos], state.goals[:], state.boxes[:])
            newState.cost = state.cost + 1
            newState.previous = state
            distance_remaining = furthestDistance(newState)

            newState.distance = distance_remaining + newState.cost
            maze.frontier.append(newState)        
      
    # Left Node
    currentBox = None
    nextBox = None
    if not maze.colsList[state.currentRowPos][state.currentColPos - 1].isWall:  
        for box in state.boxes:
            if box.rowPos == state.currentRowPos and box.colPos - 1 == state.currentColPos:
                currentBox = box
        for box in state.boxes:
            if box.rowPos == state.currentRowPos and box.colPos - 2 == state.currentColPos:
                nextBox = box
        if currentBox != None:
            if nextBox == None:
                currentBox.colPos -= 1
                newState = State(maze.colsList[state.currentRowPos][state.currentColPos - 1], state.goals[:], state.boxes[:])
                newState.cost = state.cost + 1
                newState.previous = state
                distance_remaining = furthestDistance(newState)
                
                newState.distance = distance_remaining + newState.cost
                #print(str(newState.currentRowPos) + "," + str(newState.currentColPos))
                maze.frontier.append(newState)
            else:
                pass
        else:  
            newState = State(maze.colsList[state.currentRowPos][state.currentColPos - 1], state.goals[:], state.boxes[:])
            newState.cost = state.cost + 1
            newState.previous = state
            distance_remaining = furthestDistance(newState)

            newState.distance = distance_remaining + newState.cost
            maze.frontier.append(newState) 
    
    # Right Node
    currentBox = None
    nextBox = None
    if not maze.colsList[state.currentRowPos][state.currentColPos + 1].isWall:  
        for box in state.boxes:
            if box.rowPos == state.currentRowPos and box.colPos + 1 == state.currentColPos:
                currentBox = box
        for box in state.boxes:
            if box.rowPos == state.currentRowPos and box.colPos + 2 == state.currentColPos:
                nextBox = box
        if currentBox != None:
            if nextBox == None:
                currentBox.colPos += 1
                newState = State(maze.colsList[state.currentRowPos][state.currentColPos + 1], state.goals[:], state.boxes[:])
                newState.cost = state.cost + 1
                newState.previous = state
                distance_remaining = furthestDistance(newState)
                
                newState.distance = distance_remaining + newState.cost
                #print(str(newState.currentRowPos) + "," + str(newState.currentColPos))
                maze.frontier.append(newState)
            else:
                pass
        else:  
            newState = State(maze.colsList[state.currentRowPos][state.currentColPos + 1], state.goals[:], state.boxes[:])
            newState.cost = state.cost + 1
            newState.previous = state
            distance_remaining = furthestDistance(newState)

            newState.distance = distance_remaining + newState.cost
            maze.frontier.append(newState) 
        
    printedFrontier = []
    for state in maze.frontier:
        printedFrontier.append(str(newState.currentRowPos) + "," + str(newState.currentColPos))
    print(printedFrontier)
    maze.printMaze()
    input("Press enter to continue")

## Goal Test

In [55]:
def goalTest(goals):      
    if goals == []:
        return True
    else:
        return False

def sokobanGoalTest(goals):
    count = 0
    for goal in goals:
        if goal.value == 'B':
            count += 1
    if count == len(goals):
        return True
    else:
        return False

In [56]:
def incrementGoalCounter(counter):
    if counter == 'a':
        return '9'
    else:
        return chr(ord(counter) - 1)

## Tests

#### Medium Maze

##### Depth-first Search

In [229]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [230]:
mediumMaze.runSearch('dfs')

0.003000497817993164
Nodes Expanded: 167
Nodes in Path: 117
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %           %           %  ..1%
% %%%% %% % % %%%%% %%%%% %%% % %%%% %% %%% %% %% %%% % %.% %
% %     %   %     %       %   %       %     %   % % %   %.% %
% % %%% % %%%% %% %%%%%%%%% %%%%% %%% % %%% % % % % %%%%%.% %
% %   %   %       %       %       %   %   % % % %     %...% %
% % %%% %%% %%% %%% % %%% %%%% %% % %%% % %%% % %%% %%%.%%% %
%   %     %     %   % %         % % %   %     %   % %...% % %
% %%% %%% %%%%% %%%%% % %%% %%% %%%%% %%%%%% %%%% %%%.%%% % %
%     %       %         % % %         %...%     %  ...%   % %
% %%%%% %%%%% %%%%% %%% % % % %%%%%%%%%.%.% %%% %%%.%%% %%% %
% %   % %   %         % %   % %     %  .%.% %.......    %   %
% % %%% % %%% %%%%% % % %%%%% % % %%% %.%.%%%.%%%%% %%%%% %%%
%   %   % %       % % %   %   % %   % %.%.....%     %   % % %
% %%% %%% % % %%% %%% %%% % %%% %%% % %.%%%%%%%%% %%% % % % %
% %   %   

##### Breadth-first Search

In [231]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [232]:
mediumMaze.runSearch('bfs')

0.011000633239746094
Nodes Expanded: 609
Nodes in Path: 95
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %     ....  %...........%....1%
% %%%% %% % % %%%%% %%%%% %%% % %%%%.%%.%%%.%% %% %%%.%.% % %
% %     %   %     %       %   %  .... %.....%   % % %...% % %
% % %%% % %%%% %% %%%%%%%%% %%%%%.%%% % %%% % % % % %%%%% % %
% %   %   %       %       %   ....%   %   % % % %     %   % %
% % %%% %%% %%% %%% % %%% %%%%.%% % %%% % %%% % %%% %%% %%% %
%   %     %     %   % %........ % % %   %     %   % %   % % %
% %%% %%% %%%%% %%%%% %.%%% %%% %%%%% %%%%%% %%%% %%% %%% % %
%     %.......%    .....% % %         %   %     %     %   % %
% %%%%%.%%%%%.%%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %
% %   %.%   %.......  % %   % %     %   % % %           %   %
% % %%%.% %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%
%   %...% %       % % %   %   % %   % % %     %     %   % % %
% %%%.%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %
% %...%   %

##### Greedy Best-first Search

In [233]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [234]:
mediumMaze.runSearch('gbfs')

0.007000446319580078
Nodes Expanded: 104
Nodes in Path: 95
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %     ....  %...........%....1%
% %%%% %% % % %%%%% %%%%% %%% % %%%%.%%.%%%.%% %% %%%.%.% % %
% %     %   %     %       %   %  .... %.....%   % % %...% % %
% % %%% % %%%% %% %%%%%%%%% %%%%%.%%% % %%% % % % % %%%%% % %
% %   %   %       %       %   ....%   %   % % % %     %   % %
% % %%% %%% %%% %%% % %%% %%%%.%% % %%% % %%% % %%% %%% %%% %
%   %     %     %   % %........ % % %   %     %   % %   % % %
% %%% %%% %%%%% %%%%% %.%%% %%% %%%%% %%%%%% %%%% %%% %%% % %
%     %.......%    .....% % %         %   %     %     %   % %
% %%%%%.%%%%%.%%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %
% %   %.%   %.......  % %   % %     %   % % %           %   %
% % %%%.% %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%
%   %...% %       % % %   %   % %   % % %     %     %   % % %
% %%%.%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %
% %...%   %

##### A* Search

In [235]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [236]:
mediumMaze.runSearch('astar')

0.02200150489807129
Nodes Expanded: 332
Nodes in Path: 95
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %           %...........%....1%
% %%%% %% % % %%%%% %%%%% %%% % %%%% %% %%%.%% %% %%%.%.% % %
% %     %   %     %       %   %       %.....%   % % %...% % %
% % %%% % %%%% %% %%%%%%%%% %%%%% %%% %.%%% % % % % %%%%% % %
% %   %   %       %       %       %   %.  % % % %     %   % %
% % %%% %%% %%% %%% % %%% %%%% %% % %%%.% %%% % %%% %%% %%% %
%   %     %     %   % %.........% % %...%     %   % %   % % %
% %%% %%% %%%%% %%%%% %.%%% %%%.%%%%%.%%%%%% %%%% %%% %%% % %
%     %.......%    .....% % %  .......%   %     %     %   % %
% %%%%%.%%%%%.%%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %
% %   %.%   %.......  % %   % %     %   % % %           %   %
% % %%%.% %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%
%   %...% %       % % %   %   % %   % % %     %     %   % % %
% %%%.%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %
% %...%   % 

#### Big Maze

##### Depth-first Search

In [237]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [238]:
bigMaze.runSearch('dfs')

0.011000871658325195
Nodes Expanded: 302
Nodes in Path: 241
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %.......%...% %     %   %   %   %    .........% %   %.........%.....%   %
%   %.%%%.% %%%.%.%.% % % %%% % % % % %%% %%%.%% %%%%.% % % %.% %%%%%.%.%%%.%%% %
%   %.....% % %...%.%   %   % % % % % %   % %.%     %.%   %  .%     %...% %...  %
%%% %%%% %% % %  %%.%% %% % %%% % % % % %%% %.%%%%% %.% %%%%%.%...% % %%% % %.% %
%   % %     %     %.....% %   %   % % %     %.....% %.%     %...%.......%   %.% %
%   % % % %   % % %.....% % % %%% % % %%%%% %%%%%.% %.%%%%% % % % %%% %.%%% %.%%%
% %   % % %   % % %.... %   %     % % ......%.....% %.....%...% %     %...%  ...%
% %%% % %%% % %%% % %..%% %%% %%% % %%.%%%%.%.%%%%% %.....%.%...%% %% %%%.% %%%.%
% %       % % %   % %...% %.....% % ... %  ...%      ...  %.%  .....% %...% %...%
% % %%%%% % % % %%% % %.%%%.%%%.% %%....% %%%%% %%% %%%.%%%.% %%%%%.%%%.%%% %.% %
%   %   %   % %     % %.....%  .......

##### Breadth-first Search

In [239]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [240]:
bigMaze.runSearch('bfs')

0.02300119400024414
Nodes Expanded: 1250
Nodes in Path: 149
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %.......%...% %     %   %   %   %             % %   %         %     %   %
%   %.%%%.% %%%.%.%.% % % %%% % % % % %%% %%% %% %%%% % % % % % %%%%% % %%% %%% %
%   %.....% % %...%.%   %   % % % % % %   % % %     % %   %   %     %   % %     %
%%% %%%% %% % %  %%.%% %% % %%% % % % % %%% % %%%%% % % %%%%% %   % % %%% % % % %
%   % %     %     %.    % %   %   % % %     %     % % %     %   %       %   % % %
%   % % % %   % % %.    % % % %%% % % %%%%% %%%%% % % %%%%% % % % %%% % %%% % %%%
% %   % % %   % % %...  %   %     % %       %     % %     %   % %     %   %     %
% %%% % %%% % %%% % %. %% %%% %%% % %% %%%% % %%%%% %     % %   %% %% %%% % %%% %
% %       % % %   % %...% %.....% %     %     %.........  % %       % %   % %   %
% % %%%%% % % % %%% % %.%%%.%%%.% %%    % %%%%%.%%% %%%.%%% % %%%%% %%% %%% % % %
%   %   %   % %     % %.....%...      

##### Greedy Best-first Search

In [241]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [242]:
bigMaze.runSearch('gbfs')

0.04300236701965332
Nodes Expanded: 288
Nodes in Path: 223
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %       %   % %     %   %   %   %             % %   %         %     %   %
%   %.%%% % %%% % % % % % %%% % % % % %%% %%% %% %%%% % % % % % %%%%% % %%% %%% %
%   %.... % % %   % %   %   % % % % % %   % % %     % %   %   %     %   % %     %
%%% %%%%.%% % %  %% %% %% % %%% % % % % %%% % %%%%% % % %%%%% %   % % %%% % % % %
%   % % ....%     %     % %   %   % % %     %     % % %     %   %       %   % % %
%   % % % %...% % %     % % % %%% % % %%%%% %%%%% % % %%%%% % % % %%% % %%% % %%%
% %   % % %  .% % %     %   %     % %       %     % %     %   % %     %   %     %
% %%% % %%% %.%%% % %  %% %%% %%% % %% %%%% % %%%%% %     % %   %% %% %%% % %%% %
% %       % %.%   % %   % %     % %     %     %           % %       % %   % %   %
% % %%%%% % %.% %%% % % %%% %%% % %%    % %%%%% %%% %%% %%% % %%%%% %%% %%% % % %
%   %   %   %.%     % %     %          

##### A* Search

In [243]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [244]:
bigMaze.runSearch('astar')

0.12700748443603516
Nodes Expanded: 1107
Nodes in Path: 149
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %.......%...% %     %   %   %   %             % %   %         %     %   %
%   %.%%%.% %%%.%.%.% % % %%% % % % % %%% %%% %% %%%% % % % % % %%%%% % %%% %%% %
%   %.....% % %...%.%   %   % % % % % %   % % %     % %   %   %     %   % %     %
%%% %%%% %% % %  %%.%% %% % %%% % % % % %%% % %%%%% % % %%%%% %   % % %%% % % % %
%   % %     %     %.... % %   %   % % %     %     % % %     %   %       %   % % %
%   % % % %   % % %   . % % % %%% % % %%%%% %%%%% % % %%%%% % % % %%% % %%% % %%%
% %   % % %   % % %   . %   %     % %       %     % %     %   % %     %   %     %
% %%% % %%% % %%% % % .%% %%% %%% % %% %%%% % %%%%% %     % %   %% %% %%% % %%% %
% %       % % %   % % ..% %.....% %     %     %.........  % %       % %   % %   %
% % %%%%% % % % %%% % %.%%%.%%%.% %%    % %%%%%.%%% %%%.%%% % %%%%% %%% %%% % % %
%   %   %   % %     % %.....%...      

#### Open Maze

##### Depth-first Search

In [245]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [246]:
openMaze.runSearch('dfs')

0.01000070571899414
Nodes Expanded: 385
Nodes in Path: 218
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P............%
%                     %.............%
%                     %.............%
%                     %.............%
%                     %.............%
%                     %%%%%%........%
%                          %........%
%                          %........%
%                          %........%
%..........................%........%
%........            ......%........%
%........%%%%%%%%%%%%...............%
%........%                          %
%........%                          %
%........%                          %
%........%                          %
%........%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


##### Breadth-first Search

In [247]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [248]:
openMaze.runSearch('bfs')

0.014000892639160156
Nodes Expanded: 524
Nodes in Path: 46
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %......       %
%                     %%%%%%.       %
%                          %.       %
%                          %.       %
%                          %.       %
%                          %.       %
%       ...................%.       %
%       .%%%%%%%%%%%%     ...       %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


##### Greedy Best-first Search

In [249]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [250]:
openMaze.runSearch('gbfs')

0.023001432418823242
Nodes Expanded: 154
Nodes in Path: 46
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %......       %
%                     %%%%%%.       %
%                          %.       %
%                          %.       %
%                          %.       %
%                          %.       %
%       ..............     %.       %
%       .%%%%%%%%%%%%........       %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


##### A* Search

In [251]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [252]:
openMaze.runSearch('astar')

0.0630035400390625
Nodes Expanded: 220
Nodes in Path: 46
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P....        %
%                     %    .        %
%                     %    .        %
%                     %    .        %
%                     %    ..       %
%                     %%%%%%.       %
%                          %.       %
%                          %.       %
%                          %.       %
%                          %.       %
%       ..............     %.       %
%       .%%%%%%%%%%%%........       %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


## Part 1.2

#### Tiny Search

##### Breadth-first Search

In [253]:
filename = "inputs/tinySearch.txt"
tinySearch = Maze(filename)

In [254]:
tinySearch.runSearch('bfs')

1.911109209060669
Nodes Expanded: 43527
Nodes in Path: 51
%%%%%%%%%%
%b..%.3..%
%.%c%.%%.%
%.% ..2%4%
%.a%P%  .%
%9  1 .6.%
%.%%%%.%.%
%8.7...%5%
%%%%%%%%%



##### A* Search

In [186]:
filename = "inputs/tinySearch.txt"
tinySearch = Maze(filename)

In [187]:
tinySearch.runSearch('astarv2')

3.5092005729675293
Nodes Expanded: 6616
Nodes in Path: 51
%%%%%%%%%%
%b..%.3..%
%.%c%.%%.%
%.%  .2%4%
%.a%P%. .%
%9  1..6.%
%.%%%%.%.%
%8.7...%5%
%%%%%%%%%


#### Small Search

##### A* Search

In [188]:
filename = "inputs/smallSearch.txt"
smallSearch = Maze(filename)

In [189]:
smallSearch.runSearch('astarv2')

KeyboardInterrupt: 

#### Medium Search

##### A* Search

In [118]:
filename = "inputs/mediumSearch.txt"
mediumSearch = Maze(filename)

In [ ]:
mediumSearch.runSearch('astarv2')

# Part 2

In [282]:
filename = "inputs/input1.txt"
input1 = Maze(filename)
input1.printMaze()

%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%


In [283]:
input1.runSokobanSearch('bfs')

['5,1', '5,1']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['2,1', '2,1', '2,1']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['4,1', '4,1', '4,1']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['1,2', '1,2', '1,2', '1,2']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['4,1', '4,1', '4,1', '4,1', '4,1']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['5,1', '5,1', '5,1', '5,1', '5,1', '5,1']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['1,3', '1,3', '1,3', '1,3', '1,3', '1,3']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['1,2', '1,2', '1,2', '1,2', '1,2', '1,2', '1,2']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['4,1', '4,1', '4,1', '4,1', '4,1']
%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
Press enter to continue
['1,4', '1,4', '1,4', '1,4', '1,4', '1,4']
%%%%%%
%    %

KeyboardInterrupt: 